Farmlink
========
**Data Cleaning**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
df = pd.read_csv(r"FarmlinkMainData.csv")
df = df.drop(10704) # just the column names again, not sure why this is here
df.head()

,id,status,Sends,Opens,Clicks,lifetodategiving,yeartodategiving,lastgiftamount,lastgiftdate,ContactType,MonetaryTier,DonorStatus,IndividualDonorType,Tier3Classification
0,89ce7fad62fd5b95aaadaa1edb786cc0,subscribed,112,3,2,200,0,200,4/23/2020,Donor,Diamond,SingleDonor,Single Donor,Single Donor
1,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,4/23/2020,Donor,Gold,SingleDonor,Single Donor,Single Donor
2,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,4/23/2020,Donor,Gold,SingleDonor,Single Donor,Single Donor
3,cec49095c9aef6252fbd30aae5345792,subscribed,112,29,2,200,0,200,4/23/2020,Donor,Diamond,SingleDonor,Single Donor,Single Donor
4,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,4/23/2020,Donor,Gold,SingleDonor,Single Donor,Single Donor


In [44]:
# Type correcting --> eventually will have to one-hot-encode probably
df["Sends"] = pd.to_numeric(df["Sends"])
df["Opens"] = pd.to_numeric(df["Opens"])
df["Clicks"] = pd.to_numeric(df["Clicks"])
df["lifetodategiving"] = pd.to_numeric(df["lifetodategiving"])
df["yeartodategiving"] = pd.to_numeric(df["yeartodategiving"])
df["lastgiftamount"] = pd.to_numeric(df["lastgiftamount"])
df["lastgiftdate"] = pd.to_datetime(df["lastgiftdate"])
df.dtypes

id                             object
status                         object
Sends                           int64
Opens                           int64
Clicks                          int64
lifetodategiving                int64
yeartodategiving                int64
lastgiftamount                  int64
lastgiftdate           datetime64[ns]
ContactType                    object
MonetaryTier                   object
DonorStatus                    object
IndividualDonorType            object
Tier3Classification            object
dtype: object

In [59]:
# Create `days_since_gift` column for each gift --> number of days from gift to when data was collected
df["days_since_gift"] = df["lastgiftdate"].max() - df["lastgiftdate"]

# Create `days_since_prev_gift` column --> number of days since user had previously given (better way to do this? probably)
# TODO fix --> gives only NaN values
df["days_since_prev_gift"] = pd.Series(np.nan, index=np.arange(df.shape[0]))
for index, row in df.iterrows():
    curr_date = row["lastgiftdate"]
    id_filtered_df = df[df["id"] == row["id"]]
    prev_dates = id_filtered_df[id_filtered_df.sort_values(by="lastgiftdate")["lastgiftdate"] < curr_date]
    if prev_dates.shape[0] != 0:
        df["days_since_prev_gift"][index] = prev_dates["lastgiftdate"][0]

df.head()

,id,status,Sends,Opens,Clicks,lifetodategiving,yeartodategiving,lastgiftamount,lastgiftdate,ContactType,MonetaryTier,DonorStatus,IndividualDonorType,Tier3Classification,days_since_gift,days_since_prev_gift
0,89ce7fad62fd5b95aaadaa1edb786cc0,subscribed,112,3,2,200,0,200,2020-04-23,Donor,Diamond,SingleDonor,Single Donor,Single Donor,866 days,NaN
1,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,2020-04-23,Donor,Gold,SingleDonor,Single Donor,Single Donor,866 days,NaN
2,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,2020-04-23,Donor,Gold,SingleDonor,Single Donor,Single Donor,866 days,NaN
3,cec49095c9aef6252fbd30aae5345792,subscribed,112,29,2,200,0,200,2020-04-23,Donor,Diamond,SingleDonor,Single Donor,Single Donor,866 days,NaN
4,778fdf5a21efed6e372e0e0bc159f92d,subscribed,112,112,3,50,0,50,2020-04-23,Donor,Gold,SingleDonor,Single Donor,Single Donor,866 days,NaN


sorry i couldn't get more done, i've been home for the week and it's been a bit hectic. i'm generally pretty free over spring break so i'll work on it more then. - athan